# Constantes

In [28]:
PLAIN_TEXT_FILES_PATH = "lemmatized/plain-text/*.txt"
CHECKSUM_PATH = "./checksum-xmls.csv"
FORCE_RELEMMATIZE = False # Force relemmatization
DOWNLOAD_MODELS = False # Update the model files

# Nettoyage et upgrade

Potentiellement besoin de :

```shell
!pip install https://github.com/PonteIneptique/pie/archive/improvement/AttentionDecoder.predict_max.zip#egg=nlp_pie --upgrade
!pip install --upgrade https://github.com/hipster-philology/nlp-pie-taggers/archive/feature/hashlist.zip#egg
```

In [11]:
!rm ./lemmatized/plain-text/*-pie*.txt
#!pip install --upgrade https://github.com/hipster-philology/nlp-pie-taggers/archive/feature/hashlist.zip#egg
#!pip install --upgrade https://github.com/PonteIneptique/pie/archive/improvement/AttentionDecoder.predict_max.zip#egg=nlp_pie

rm: cannot remove './lemmatized/plain-text/*-pie*.txt': No such file or directory


# Récupération des textes à lemmatiser

## Lecture du fichier des checksum

In [25]:
from pie_extended.utils.hashcheck import md5sum, check_checksum_from_file, read_checksum_csv
import os.path

target_files = glob.glob(PLAIN_TEXT_FILES_PATH)
existing_files = {
    os.path.abspath(target): md5sum(target)
    for target in target_files
}

former_checksums = read_checksum_csv(CHECKSUM_PATH)
new_input = [target for target in existing_files if target not in former_checksums]
changed_input, _ = check_checksum_from_file(CHECKSUM_PATH)


print(f"{len(former_checksums)} former source file")
print(f"{len(target_files)} target files")
print(f"{len(changed_input)} changed input files detected")
print(f"{len(new_input)} new input files detected")

652 former source file
652 target files
0 changed input files detected
0 new input files detected


## Génération de la liste des fichiers à produire

In [30]:
texts_to_lemmatize = []

if FORCE_RELEMMATIZE:
    texts_to_lemmatize = list(existing_files.keys())
else:
    texts_to_lemmatize = new_input + changed_input
    
print(f"{len(texts_to_lemmatize)} files to lemmatize")

0 files to lemmatize


# Lemmatisation

## Initialisation du tagger

In [27]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

if DOWNLOAD_MODELS:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=128, device="cuda", model_path=None)

## Taggage

In [31]:
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor
import glob
import tqdm

for file in tqdm.tqdm(texts_to_lemmatize):
    try:
        if "-pie" not in file:
            
            iterator, processor = get_iterator_and_processor()
            tagger.tag_file(file, iterator=iterator, processor=processor)
    except Exception as E:
        print(file)
        print(E)


0it [00:00, ?it/s]


## Debug

In [32]:
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor

iterator, processor = get_iterator_and_processor()
file = "lemmatized/plain-text/urn:cts:latinLit:stoa0275.stoa006.opp-lat1.txt"
#with open(file) as f:
#    print(f.read())

#tagger.tag_file(file, iterator=iterator, processor=processor)


## Déplacement des fichiers dans le dossier TSV

In [33]:
!mkdir -p lemmatized/tsv
!mv lemmatized/plain-text/*-pie.txt lemmatized/tsv/

mv: cannot stat 'lemmatized/plain-text/*-pie.txt': No such file or directory


# Update de l'index des fichiers à lemmatiser

In [34]:
## ToDo When there is a new update of the corpus